In [1]:
import sys
import os
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, parent_dir)

import torch
import numpy as np
save_directory = "/local/scratch/Data/TROPHY/numpy/"
from Sample import Sample
import numpy as np
from torch.utils.data import DataLoader, TensorDataset 
import torch.nn as nn
import torch.optim as optimise
samples = np.load(save_directory + "samples.npy", allow_pickle = True)
normalised_data = np.load(save_directory + "sigmoid_normalised_data.npy")
classifications = np.load(save_directory + "classification.npy")

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
print(len(samples))
print(samples[77].end_index)
cutoff_index = samples[77].end_index

90
1043928


In [4]:
train_data = torch.tensor(normalised_data[:cutoff_index], dtype=torch.float32).cpu() 
test_data = torch.tensor(normalised_data[cutoff_index:], dtype=torch.float32).cpu()
train_classes = torch.tensor(classifications[:cutoff_index] - 1, dtype=torch.long).cpu()
test_classes = torch.tensor(classifications[cutoff_index:] - 1, dtype=torch.long).cpu()

print(train_classes[10])
train_dataset = TensorDataset(train_data, train_classes)
test_dataset = TensorDataset(test_data, test_classes)
train_loader = DataLoader(train_dataset, batch_size=9600, shuffle=True, num_workers=14, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=9600, num_workers=14, pin_memory=True)

class MLPModel(nn.Module):
    def __init__(self, input_size, l1_size, l2_size, l3_size, l4_size, l5_size, output_size):
        super(MLPModel, self).__init__()
        self.layer1 = nn.Linear(input_size, l1_size)
        self.layer2 = nn.Linear(l1_size, l2_size)
        self.layer3 = nn.Linear(l2_size, l3_size)
        self.layer4 = nn.Linear(l3_size, l4_size)
        self.layer5 = nn.Linear(l4_size, l5_size)
        self.layer6 = nn.Linear(l5_size, output_size)
        self.reLU = nn.ReLU()

    def forward(self, x):
        x = self.reLU(self.layer1(x))
        x = self.reLU(self.layer2(x))
        x = self.reLU(self.layer3(x))
        x = self.reLU(self.layer4(x))
        x = self.reLU(self.layer5(x))
        x = self.layer6(x)
        return x

model = MLPModel(801, 10000, 5000, 1000, 100, 20, 4).to(device)
criterion = nn.CrossEntropyLoss()
optimiser = optimise.Adam(model.parameters(), lr=0.0001)


tensor(2)


In [5]:
print(device)
epochs = 400
for epoch in range(epochs):
    model.train()
    scaler = torch.amp.GradScaler()
    for X_batch, Y_batch in train_loader:
        X_batch, Y_batch = X_batch.to(device, non_blocking=True), Y_batch.to(device, non_blocking=True)
        optimiser.zero_grad()
        # with torch.cuda.amp.autocast():
        outputs = model(X_batch)
        loss = criterion(outputs, Y_batch)
        # scaler.scale(loss).backward()
        # scaler.step(optimiser)
        # scaler.update()
        outputs = model(X_batch)
        loss = criterion(outputs, Y_batch)
        loss.backward()
        optimiser.step()

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for X_batch, Y_batch in test_loader:
            X_batch, Y_batch = X_batch.to(device, non_blocking=True), Y_batch.to(device, non_blocking=True)
            outputs = model(X_batch)
            loss = criterion(outputs, Y_batch)
            val_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Training Loss: {loss.item()}, Validation Loss: {val_loss / len(test_loader)}")

print("Training complete")

cuda
Epoch 1/400, Training Loss: 2.467228889465332, Validation Loss: 1.0644907588139176
Epoch 2/400, Training Loss: 2.170783042907715, Validation Loss: 0.9560940559022129
Epoch 3/400, Training Loss: 2.502584457397461, Validation Loss: 1.0120691158808768
Epoch 4/400, Training Loss: 3.1601338386535645, Validation Loss: 1.1094798408448696
Epoch 5/400, Training Loss: 3.18863582611084, Validation Loss: 1.0846811840310693
Epoch 6/400, Training Loss: 3.634798526763916, Validation Loss: 1.2941246693953872
Epoch 7/400, Training Loss: 1.9039463996887207, Validation Loss: 0.8917692806571722
Epoch 8/400, Training Loss: 2.747490644454956, Validation Loss: 1.1534282159991562
Epoch 9/400, Training Loss: 3.1158368587493896, Validation Loss: 1.2878677789121866
Epoch 10/400, Training Loss: 3.319990873336792, Validation Loss: 1.322663526982069
Epoch 11/400, Training Loss: 3.3242790699005127, Validation Loss: 1.4002099772915244
Epoch 12/400, Training Loss: 3.5290253162384033, Validation Loss: 1.4938866449

KeyboardInterrupt: 